The contents of this course including lectures, labs, homework assignments, and exams have all been adapted from the [Data 8 course at University California Berkley](https://data.berkeley.edu/education/courses/data-8). Through their generosity and passion for undergraduate education, the Data 8 community at Berkley has opened their content and expertise for other universities to adapt in the name of undergraduate education.

In [ ]:
!pip install datascience
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Chapter 6: Tables
- There are too many Table functions to list here.  
- [Table documentation](http://data8.org/datascience/tables.html)
- Here are some functions to get us started
    - Table.read_table(file_name) to load .csv file as a Table
    - Table.with_columns('Label_1', Values_1, 'Label_2', Values_2, etc.)) to create Table
    - Table.columns('Label') to get array of values from Label index
    - .sum('Label'), .min('Label'), .max('Label') of values from Label index
    - .drop('Label') to drop Label from Table
    - Table.sort('Label') sorts the rows in increasing order
    - Table.take('row_numbers') keeps the numbered row (indexing starts at 0)
    - Table.where('Label', condition) keeps rows where condition is satisfied.

## Discussion Question: NBA Salaries

Create an array containing the names of all point guards (PG) who make more than 15M/year.

In [ ]:
nba = Table.read_table('nba_salaries.csv')
nba

In [ ]:
nba = nba.relabeled(3, 'SALARY').drop('TEAM')
nba.show(3)

In [ ]:
# Create an array containing the names of all point guards (PG) who make more thatn %15M/year.
guards = nba.where('POSITION', 'PG')
guards.where('SALARY', are.above(15)).column('PLAYER')

## Attributes
- Values in a column of a table should be of the same type and be comparable in some way
- Numerical
    * Numerical values are ordered
    * Difference are meaningful
- Categorical
    * May or may not be ordered
- Numerical data can be categorical
    - Census gender data is coded as 0, 1, or 2.  
    - It would be meaningless to perform arthimitic on these data
    - SEX or Gender data is categorical eventhough it is numerical

## Census data
- Column SEX 1 == Male, 2 == Female, 0 == Total (male + female)
- Column POPESTIMATE2010 is 7/1/2020 population estimate
- Column AGE == 999 == total of all ages
- Column AGE is not necessarialy comparible because of 999 value


# Census Demo
Create a table that shows total population of each age bracket in 2010 and 2014

In [ ]:
full = Table.read_table('nc-est2014-agesex-res.csv')
full

In [ ]:
partial = full.select('SEX', 'AGE', 'CENSUS2010POP', 'POPESTIMATE2014')
partial.show(4)

In [ ]:
simple = partial.relabeled(2, '2010').relabeled(3, '2014')
simple.show(4)

In [ ]:
simple.sort('AGE')

In [ ]:
simple.sort('AGE', descending=True)

In [ ]:
no_999 = simple.where('AGE', are.below(999))
everyone = no_999.where('SEX', 0).drop('SEX')

In [ ]:
everyone

## Visualization ##

In [ ]:
everyone.plot('AGE', '2014')

# Chapter 7: Visualization
Tables are a powerful way of organizing data.  However, large tables are difficult to digest so it is better to graph the data in a table.

## Let's look at the Census data again

In [ ]:
full = Table.read_table('nc-est2014-agesex-res.csv')
full

In [ ]:
# Keep only the columns we care about
partial = full.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2014')
partial

In [ ]:
# Make things easier to read
simple = partial.relabeled(2, '2010').relabeled(3, '2014')
simple

In [ ]:
# Sort by age
simple.sort('AGE')

In [ ]:
# Sort by age (another way)
simple.sort('AGE', descending=True)

## Line Plots ##

In [ ]:
# Remove the age totals
no_999 = simple.where('AGE', are.below(999))

In [ ]:
# Remove male and female (keep only combined)
everyone = no_999.where('SEX', 0).drop('SEX')

In [ ]:
everyone

In [ ]:
everyone.plot('AGE', '2010')

In [ ]:
# ^^ That plot should be labeled! Here are 3 ways to label it:

In [ ]:
# US Population  <--- Just add a comment

everyone.plot('AGE', '2010')

In [ ]:
everyone.plot('AGE', '2010')
print('US Population')  # <--- Print out what it is

In [ ]:
# Add title to plot
everyone.plot('AGE', '2010')
plots.title('US Population');

In [ ]:
# Age distribution for two different years
everyone.plot('AGE')

## Males and Females in 2014 ##

In [ ]:
# Let's compare male and female counts per age
males = no_999.where('SEX', 1).drop('SEX')
females = no_999.where('SEX', 2).drop('SEX')

In [ ]:
pop_2014 = Table().with_columns(
    'Age', males.column('AGE'),
    'Males', males.column('2014'),
    'Females', females.column('2014')
)
pop_2014

In [ ]:
pop_2014.plot('Age')

In [ ]:
# Calculate the percent female for each age
total = pop_2014.column('Males') + pop_2014.column('Females')
pct_female = pop_2014.column('Females') / total * 100
pct_female

In [ ]:
# Round it to 3 so that it's easier to read
pct_female = np.round(pct_female, 3)
pct_female

In [ ]:
# Add female percent to our table
pop_2014 = pop_2014.with_column('Percent female', pct_female)
pop_2014

In [ ]:
pop_2014.plot('Age', 'Percent female')

In [ ]:
# ^^ Look at the y-axis! Trend is not as dramatic as you might think.
pop_2014.plot('Age', 'Percent female')
plots.ylim(0, 100);

## Scatter Plots ##

In [ ]:
# Actors and their highest grossing movies
actors = Table.read_table('actors.csv')
actors

In [ ]:
actors.scatter('Number of Movies', 'Total Gross')

In [ ]:
actors.plot('Number of Movies', 'Total Gross')

## When to use line vs. scatter

* use line plots for sequential data if:
    * your x-axis has order
    * differences in y values are meaningful
    * there's only on y-value for each x-value
    * X values are usually time or distance
* use scatter plots for non-sequential data
    * When looking for associations

In [ ]:
actors.scatter('Number of Movies', 'Average per Movie')

In [ ]:
actors.where('Average per Movie', are.above(400))

## Bar Charts ##

## C-3P0
#### Anthony Daniels, actor

<img src="C3P0.png" style="width: 200px;"/>

https://en.wikipedia.org/wiki/C-3PO

In [ ]:
# Highest grossing movies as of 2017
top_movies = Table.read_table('top_movies_2017.csv')
top_movies

In [ ]:
top10_adjusted = top_movies.take(np.arange(10))
top10_adjusted

In [ ]:
# Convert to millions of dollars for readability
millions = np.round(top10_adjusted.column('Gross (Adjusted)') / 1000000, 3)
top10_adjusted = top10_adjusted.with_column('Millions', millions)
top10_adjusted

In [ ]:
# A line plot doesn't make sense here: don't do this!
top10_adjusted.plot('Year', 'Millions')

In [ ]:
top_movies = top_movies.with_column('Millions', np.round(top_movies.column('Gross')/1000000,3))
top_movies.take(np.arange(10)).barh('Title', 'Millions')

In [ ]:
top10_adjusted.barh('Title', 'Millions')

## Generate the bar chart shown for the 10 highest grossing movies (non-adjusted)

<img src = 'barchartmovies.png'>

In [ ]:
top10 = top_movies.sort('Gross', descending = True).take(np.arange(10))
gross_millions = np.round(top10.column('Gross') / 1000000, 3)
top10 = top10.with_column('Millions', gross_millions)
top10_year = top10.with_column('Age', 2021 - top10.column('Year'))

In [ ]:
top10_year

In [ ]:

top10_year.barh('Title', 'Age')

## What's wrong with the the figure below? 
### Remake a horizontal bar plot with the data on the y-axis going to zero.

<img src='bush_tax_cut.jpg'>

https://www.kdnuggets.com/2012/12/taking-misleading-statistics-to-a-new-level.html

In [ ]:
bush = Table().with_columns('Date', ['Now', 'Jan 1, 2013'],
                           'Tax Rate', [35, 39.6])
bush.barh('Date')

## Catagorical Distributions
* Bar Charts are commonly used to visualize catergorical distributions
* One axis is categorical, one numerical
* The distribution of a variable (a column e.g. studios) describes the frequencies of its different values.
* The group function counts the number of rows for each value in the column. (e.g. the number of movies released by each studio
* Bar charts can display the distribution of a categorical bariable (e.g. studios)
    * One bar for each category
    * Length of bar is the count of individuals in that category
    * You can choose the order of the bars

In [ ]:
top_movies = Table.read_table('top_movies_2017.csv')
top_movies

In [ ]:
studios = top_movies.select('Studio')
studios

In [ ]:
studio_distribution = studios.group('Studio')

In [ ]:
studio_distribution

In [ ]:
sum(studio_distribution.column('count'))

In [ ]:
studio_distribution.barh('Studio')

In [ ]:
studio_distribution.sort('count', descending=True).barh('Studio')

## Numerical Distribution
### Binning Numerical Values
Binning is counting the number of numberical values that lie within ranges, called bins
* Bins are defined by their lower bounds (inclusive)
* The upper bound is the lower bound of the next bin
* Counts all the values that are greater than or equal to the value in the bin but less than the next value in the bin
* The last bin is always zero because there is no upper bound

In [ ]:
ages = 2021 - top_movies.column('Year')
top_movies = top_movies.with_column('Age', ages)

In [ ]:
top_movies

In [ ]:
min(ages), max(ages)

In [ ]:
my_bins = make_array(0, 5, 10, 15, 25, 40, 65, 100)

In [ ]:
binned_data = top_movies.bin('Age', bins = my_bins)
binned_data

In [ ]:
sum(binned_data.column('Age count'))

In [ ]:
top_movies.bin('Age', bins = np.arange(0, 101, 25))

In [ ]:
top_movies.bin('Age', bins = np.arange(0, 100, 10))

In [ ]:
top_movies.where('Age', are.above(70)).show(11)

## Histograms ##

In [ ]:
my_bins

In [ ]:
binned_data

In [ ]:
# Let's make our first histogram!
top_movies.hist('Age', bins = my_bins, unit = 'Year')

In [ ]:
# Let's try equally spaced bins instead.
top_movies.hist('Age', bins = np.arange(0, 110, 10), unit = 'Year')

In [ ]:
# Let's try not specifying any bins!
top_movies.hist('Age', unit='Year')

## Histogram height
area = height*width

area = percent

height = percent/width

Let's calcualte percent first

In [ ]:
# Add a column containing what percent of movies are in each bin
binned_data = binned_data.with_column(
    'Percent', 100*binned_data.column('Age count')/200)

In [ ]:
binned_data

### Question: What is the height of the [40, 65] bin?

In [ ]:
# Step 1: Calculate % of movies in the [40, 65) bin
percent = binned_data.where('bin', 40).column('Percent').item(0)
percent

In [ ]:
# Step 2: Calculate the width of the 40-65 bin
width = 65 - 40

In [ ]:
# Step 3: Area of rectangle = height * width
#         --> height = percent / width
height = percent / width
height

In [ ]:
top_movies.hist('Age', bins = my_bins, unit = 'Year')

In [ ]:
top_movies.hist('Age', bins = np.arange(0, 110, 10), unit = 'Year')

In [ ]:
top_movies.hist('Age', bins = make_array(1, 10, 100), unit = 'Year')


## Differences Between Bar Charts and Histograms

* Bar charts display one quantity per category. They are often used to display the distributions of categorical variables. Histograms display the distributions of quantitative variables.
* All the bars in a bar chart have the same width, and there is an equal amount of space between consecutive bars. The bars of a histogram can have different widths, and they are contiguous.
* The lengths (or heights, if the bars are drawn vertically) of the bars in a bar chart are proportional to the value for each category. The heights of bars in a histogram measure densities; the areas of bars in a histogram are proportional to the numbers of entries in the bins.



## What is wrong with this picture?

<img src = ipad.png style="width: 500px;">
caption: The new battery is 70% larger

https://gizmodo.com/holy-f-ck-the-new-ipad-has-a-gigantic-70-percent-large-5893738

## Chart Distribution Review

#### Bar Chart
* Distribution of categorical variables
* Bars have equal yet arbitrary widths/spacing
* area = percent of individuals

#### Histogram Charts
* Distribution of numerical variables
* Horizontal axis is numberical bin width can be unequal
* area = percent of individuals
* height = percent/width

## Chart Review
* Scatter plot: relation between numerical variables
* Line graph: sequential data (time or distance)
* Bar chart: distribution of categorical data
* Histogram: distribution of numerical data

In [ ]:
# Create some fake weather data for a discussion question
weather = Table().with_columns('Day', np.arange(1, 366),
                              'High',  np.round(np.random.uniform(low=70, high=100, size=365),1),
                              'Low',  np.round(np.random.uniform(low=32, high=70, size=365),1),
                              'Sky Condition', np.random.choice(['Cloudy', 'Sunny'], 365, p=[0.49, 0.51]))
                
weather

### Create a chart to answer the following questions about the weather table
1) Are there more cloudy than sunny days?

2) What percentage of days have a high above 72 degrees?

3) Do hotter days have hotter nights?

In [ ]:
weather.group('Sky Condition').barh('Sky Condition')

In [ ]:
weather.hist('High', bins = make_array(0, 72, 100), unit = 'Temp')
plots.minorticks_on()
plots.grid(b=True, which='minor', color = 'k')
plots.grid(b=True, which='major', color = 'r')
weather.where('High', are.above_or_equal_to(72)).num_rows/weather.num_rows

In [ ]:
weather.select('Day' ,'High', 'Low').plot('Day')

In [ ]:
weather.scatter('Low', 'High')